In [3]:
# Dependencies
import requests
import pandas as pd
import json
from pprint import pprint
import api_keys
import time 

In [2]:
#winery set up
wine_xl = pd.read_excel("winery_list_final.xlsx")

wine_xl.head()

,OWNER NAME,OPERATING NAME,STREET,CITY,STATE,ZIP,COUNTY
0,"DASHE WINE CELLARS, LLC",DASHE CELLARS,1951 MONARCH ST STE 300,ALAMEDA,CA,94501,ALAMEDA
1,"ROCK WALL WINE COMPANY, INC.",NaN,2301 MONARCH ST SUITE #300,ALAMEDA,CA,94501,ALAMEDA
2,"EHRENBERG CELLAR, LLC",EHRENBERG CELLARS,2301 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA
3,"LA COSTA PACIFICA, INC.",URBAN LEGEND WINE,1951 MONARCH ST SUITE 300,ALAMEDA,CA,94501,ALAMEDA
4,SIMAS FAMILY VINEYARD LLC,SIMAS FAMILY VINEYARD,1951 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA


In [3]:
print(wine_xl.count())

wine_xl2 = wine_xl.dropna(how="any")

wine_xl2.count()

OWNER NAME        4949
OPERATING NAME    3657
STREET            4940
CITY              4949
STATE             4949
ZIP               4949
COUNTY            4668
dtype: int64


OWNER NAME        3465
OPERATING NAME    3465
STREET            3465
CITY              3465
STATE             3465
ZIP               3465
COUNTY            3465
dtype: int64

In [4]:
#still index to when we had a NaN 
#need to reset index
wine_xl2 = wine_xl2.reset_index()
wine_xl2.head()

,index,OWNER NAME,OPERATING NAME,STREET,CITY,STATE,ZIP,COUNTY
0,0,"DASHE WINE CELLARS, LLC",DASHE CELLARS,1951 MONARCH ST STE 300,ALAMEDA,CA,94501,ALAMEDA
1,2,"EHRENBERG CELLAR, LLC",EHRENBERG CELLARS,2301 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA
2,3,"LA COSTA PACIFICA, INC.",URBAN LEGEND WINE,1951 MONARCH ST SUITE 300,ALAMEDA,CA,94501,ALAMEDA
3,4,SIMAS FAMILY VINEYARD LLC,SIMAS FAMILY VINEYARD,1951 MONARCH ST,ALAMEDA,CA,94501,ALAMEDA
4,7,STEELTOWN WINERY LLC,BUILDING 43 WINERY,2440 MONARCH ST STE 100,ALAMEDA,CA,94501,ALAMEDA


In [5]:
wine_name = wine_xl2["OPERATING NAME"]
wine_street = wine_xl2["STREET"]
wine_city = wine_xl2["CITY"]
wine_state = wine_xl2["STATE"]
wine_zip = wine_xl2["ZIP"]
print(wine_name[0])
print(wine_street[0] + ", " + wine_city[0] + ", " + wine_state[0] + " " + str(wine_zip[0]))

DASHE CELLARS
1951 MONARCH ST STE 300, ALAMEDA, CA 94501


In [6]:
#example api call for winery
#with the list of brewery, set up a test call with yelp
#yelp api call on all brewery names, and address with city
#example use for the api call
wine_name1 = wine_name[0]
wine_location1 = wine_street[0] + ", " + wine_city[0] + ", " + wine_state[0] + " " + str(wine_zip[0])
print(wine_location1)
url = f'https://api.yelp.com/v3/businesses/search?term={wine_name1}&location={wine_location1}'
headers = {'Authorization': f"Bearer {api_keys.api_keys}"}
result2 = requests.get(url, headers=headers).json()

print(result2)


1951 MONARCH ST STE 300, ALAMEDA, CA 94501
{'businesses': [{'id': '5T8OPIIctx3wqrPwSHDDMw', 'alias': 'dashe-cellars-alameda', 'name': 'Dashe Cellars', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/PNP6fEoG4-N8Ev9BzzHSxg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/dashe-cellars-alameda?adjust_creative=nMvZa6MKHlsfUjVTbGXPcA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nMvZa6MKHlsfUjVTbGXPcA', 'review_count': 101, 'categories': [{'alias': 'venues', 'title': 'Venues & Event Spaces'}, {'alias': 'winetastingroom', 'title': 'Wine Tasting Room'}], 'rating': 4.5, 'coordinates': {'latitude': 37.780068, 'longitude': -122.31026}, 'transactions': [], 'price': '$$', 'location': {'address1': '1951 Monarch St', 'address2': 'Ste 300', 'address3': '', 'city': 'Alameda', 'zip_code': '94501', 'country': 'US', 'state': 'CA', 'display_address': ['1951 Monarch St', 'Ste 300', 'Alameda, CA 94501']}, 'phone': '+15104521800', 'display_phone': '(510) 452-1800', 

In [7]:
#interested fields from the yelp api call for wine (same as brewery)
#name, biz-type, address, city, lat, long, yelp rating, number of ratings
print(result2["businesses"][0]["name"])
print(result2["businesses"][0]["categories"][0]["title"])
print(result2["businesses"][0]["location"]["address1"])
print(result2["businesses"][0]["location"]["city"])
print(result2["businesses"][0]["location"]["state"])
print(result2["businesses"][0]["location"]["zip_code"])
print(result2["businesses"][0]["coordinates"]["latitude"])
print(result2["businesses"][0]["coordinates"]["longitude"])
print(result2["businesses"][0]["rating"])
print(result2["businesses"][0]["review_count"])

Dashe Cellars
Venues & Event Spaces
1951 Monarch St
Alameda
CA
94501
37.780068
-122.31026
4.5
101


In [8]:
print(wine_name[0])
print(wine_name[1])
print(wine_name[2])

DASHE CELLARS
EHRENBERG CELLARS
URBAN LEGEND WINE


In [15]:
#name, biz-type, address, city, lat, long, yelp rating, number of ratings

wine_name_yelp = []
wine_biz_type = []
wine_address = []
wine_yelp_city = []
wine_yelp_zip = []
wine_lat = []
wine_long = []
wine_rating = []
wine_rate_num = []

count = 0
#call in 3 different ranges for 3465 winery
#1-1000 range(0, 1001)
#1001 - 2000 range(1001, 2001)
#2001 - 3000 range(2001, 3001)
#3000 - 3465 range(3001, 3466)

for num in range(0,3465):
    wine_name2 = wine_name[num]
    wine_location2 = wine_street[num] + ", " + wine_city[num] + ", " + wine_state[num] + " " + str(wine_zip[num])
    
    print(wine_location2)
    count += 1
    print(f"Searching for: {wine_name2}")
    print("---------------------------------------")
    try:
        url1 = f'https://api.yelp.com/v3/businesses/search?term={wine_name2}&location={wine_location2}'
        headers = {'Authorization': f"Bearer {api_keys.api_keys}"}
        yelp_wine_search_results = requests.get(url1, headers=headers).json()

        print(f"Found result for {count}: {wine_name2}")
        print("---------------------------------------")

        wine_name_yelp.append(yelp_wine_search_results["businesses"][0]["name"])
        wine_biz_type.append(yelp_wine_search_results["businesses"][0]["categories"][0]["title"])
        wine_address.append(yelp_wine_search_results["businesses"][0]["location"]["address1"])
        wine_yelp_city.append(yelp_wine_search_results["businesses"][0]["location"]["city"])
        wine_yelp_zip.append(yelp_wine_search_results["businesses"][0]["location"]["zip_code"])
        wine_lat.append(yelp_wine_search_results["businesses"][0]["coordinates"]["latitude"])
        wine_long.append(yelp_wine_search_results["businesses"][0]["coordinates"]["longitude"])
        wine_rating.append(yelp_wine_search_results["businesses"][0]["rating"])
        wine_rate_num.append(yelp_wine_search_results["businesses"][0]["review_count"])
        time.sleep(1)
    except:
        print("No brewery found using addresss requirement.")
        print("Skipping....")

1951 MONARCH ST STE 300, ALAMEDA, CA 94501
Searching for: DASHE CELLARS
---------------------------------------
Found result for 1: DASHE CELLARS
---------------------------------------
2301 MONARCH ST, ALAMEDA, CA 94501
Searching for: EHRENBERG CELLARS
---------------------------------------
Found result for 2: EHRENBERG CELLARS
---------------------------------------
1951 MONARCH ST SUITE 300, ALAMEDA, CA 94501
Searching for: URBAN LEGEND WINE
---------------------------------------
Found result for 3: URBAN LEGEND WINE
---------------------------------------
1951 MONARCH ST, ALAMEDA, CA 94501
Searching for: SIMAS FAMILY VINEYARD
---------------------------------------
Found result for 4: SIMAS FAMILY VINEYARD
---------------------------------------
No brewery found using addresss requirement.
Skipping....
2440 MONARCH ST STE 100, ALAMEDA, CA 94501
Searching for: BUILDING 43 WINERY
---------------------------------------
Found result for 5: BUILDING 43 WINERY
------------------------

Found result for 42: STONY RIDGE WINERY
---------------------------------------
2400 ARROYO RD, LIVERMORE, CA 94550
Searching for: CUDA RIDGE
---------------------------------------
Found result for 43: CUDA RIDGE
---------------------------------------
503 & 511 LEISURE STREET, LIVERMORE, CA 94550
Searching for: LEISURE STREET WINERY
---------------------------------------
Found result for 44: LEISURE STREET WINERY
---------------------------------------
1960 S. LIVERMORE AVE, LIVERMORE, CA 94550
Searching for: PAGE MILL WINERY
---------------------------------------
Found result for 45: PAGE MILL WINERY
---------------------------------------
5565 TESLA RD, LIVERMORE, CA 94550
Searching for: THE STEVEN KENT WINERY
---------------------------------------
Found result for 46: THE STEVEN KENT WINERY
---------------------------------------
633 KALTHOFF COMMON, LIVERMORE, CA 94550
Searching for: ENOS FAMILY VINEYARDS
---------------------------------------
Found result for 47: ENOS FAMILY

12300 STEINER RD, PLYMOUTH, CA 95669
Searching for: SHENANDOAH VINEYARDS
---------------------------------------
Found result for 84: SHENANDOAH VINEYARDS
---------------------------------------
11001 VALLEY DR, PLYMOUTH, CA 95669
Searching for: TKC VINEYARDS
---------------------------------------
Found result for 85: TKC VINEYARDS
---------------------------------------
10590 SHENANDOAH RD, PLYMOUTH, CA 95669
Searching for: BRAY VINEYARDS
---------------------------------------
Found result for 86: BRAY VINEYARDS
---------------------------------------
11000 SHENANDOAH RD, PLYMOUTH, CA 95669
Searching for: BAIOCCHI WINERY
---------------------------------------
Found result for 87: BAIOCCHI WINERY
---------------------------------------
19890 SHENANDOAH SCHOOL RD, PLYMOUTH, CA 95669
Searching for: WILDEROTTER WINERY
---------------------------------------
Found result for 88: WILDEROTTER WINERY
---------------------------------------
12500 STEINER RD, PLYMOUTH, CA 95669
Searching for

4378 MYVALLI DR, BUTTE VALLEY, CA 95965
Searching for: STRAW HOUSE CELLARS
---------------------------------------
Found result for 124: STRAW HOUSE CELLARS
---------------------------------------
No brewery found using addresss requirement.
Skipping....
3363 HEGAN LN, CHICO, CA 95928
Searching for: BERTAGNA SON KISSED VINEYARD
---------------------------------------
Found result for 125: BERTAGNA SON KISSED VINEYARD
---------------------------------------
5900 ANITA RD, CHICO, CA 95973
Searching for: RONEY WINES
---------------------------------------
Found result for 126: RONEY WINES
---------------------------------------
No brewery found using addresss requirement.
Skipping....
630 OASIS DR, CHICO, CA 95973
Searching for: WILD OAK VINEYARD AND WINERY
---------------------------------------
Found result for 127: WILD OAK VINEYARD AND WINERY
---------------------------------------
No brewery found using addresss requirement.
Skipping....
6237 COHASSET RD, CHICO, CA 95973
Searching fo

97 EMERALD CREEK DR, MURPHYS, CA 95247
Searching for: NEWSOME-HARLOW WINERY
---------------------------------------
Found result for 163: NEWSOME-HARLOW WINERY
---------------------------------------
3468 MURPHYS GRADE RD SUITE A, MURPHYS, CA 95247
Searching for: MILLIAIRE CELLARS
---------------------------------------
Found result for 164: MILLIAIRE CELLARS
---------------------------------------
4667 FRENCH GULCH RD STE C, MURPHYS, CA 95247
Searching for: LOCKE VINEYARDS
---------------------------------------
Found result for 165: LOCKE VINEYARDS
---------------------------------------
3468 MURPHYS GRADE RD SUITE B, MURPHYS, CA 95247
Searching for: BLACK SHEEP VINTNERS
---------------------------------------
Found result for 166: BLACK SHEEP VINTNERS
---------------------------------------
No brewery found using addresss requirement.
Skipping....
5599 DEL SOL LN, SAN ANDREAS, CA 95249
Searching for: LAZZERI FAMILY VINEYARDS
---------------------------------------
Found result for 1

...

1446 INDUSTRIAL AVE, SEBASTOPOL, CA 95472
Searching for: MAGITO WINES
---------------------------------------
Found result for 3297: MAGITO WINES
---------------------------------------
2064 GRAVENSTEIN HWY, SEBASTOPOL, CA 95472
Searching for: BAD RAT RANCH
---------------------------------------
Found result for 3298: BAD RAT RANCH
---------------------------------------
11400 GRATON RD, SEBASTOPOL, CA 95472
Searching for: MARIMAR ESTATE VINEYARDS & WINERY
---------------------------------------
Found result for 3299: MARIMAR ESTATE VINEYARDS & WINERY
---------------------------------------
1600 BARLOW LN, SEBASTOPOL, CA 95472
Searching for: MCEVOY RANCH
---------------------------------------
Found result for 3300: MCEVOY RANCH
---------------------------------------
1600 BARLOW LN, SEBASTOPOL, CA 95472
Searching for: MCINTYRE VINEYARDS
---------------------------------------
Found result for 3301: MCINTYRE VINEYARDS
---------------------------------------
2064 GRAVENSTEIN HWY SUITE 

392 LONDON WAY, SONOMA, CA 95476
Searching for: AMAPOLA CREEK
---------------------------------------
Found result for 3337: AMAPOLA CREEK
---------------------------------------
990 CAVEDALE RD, SONOMA, CA 95476
Searching for: ANGRY KIWI WINES
---------------------------------------
Found result for 3338: ANGRY KIWI WINES
---------------------------------------
21684 8TH ST SUITES 440, SONOMA, CA 95476
Searching for: AUTEUR
---------------------------------------
Found result for 3339: AUTEUR
---------------------------------------
396 LONDON WAY, SONOMA, CA 95476
Searching for: B WISE VINEYARDS
---------------------------------------
Found result for 3340: B WISE VINEYARDS
---------------------------------------
21481 EIGHTH ST E STE 15, SONOMA, CA 95476
Searching for: BEDROCK WINE COMPANY
---------------------------------------
Found result for 3341: BEDROCK WINE COMPANY
---------------------------------------
21481 8TH ST E STE 25, SONOMA, CA 95476
Searching for: BELTANE RANCH
----

Found result for 3379: HAMEL FAMILY WINES
---------------------------------------
21684 E EIGHTH ST 520, SONOMA, CA 95476
Searching for: HARASZTHY FAMILY CELLARS
---------------------------------------
Found result for 3380: HARASZTHY FAMILY CELLARS
---------------------------------------
22033 BONNESS RD, SONOMA, CA 95476
Searching for: CURVARE WINERY
---------------------------------------
Found result for 3381: CURVARE WINERY
---------------------------------------
22985 BURNDALE RD, SONOMA, CA 95476
Searching for: HEIRS OF MY DREAM
---------------------------------------
Found result for 3382: HEIRS OF MY DREAM
---------------------------------------
21481 E 8TH ST UNIT 15, SONOMA, CA 95476
Searching for: MICHAEL HAVENS WINES
---------------------------------------
Found result for 3383: MICHAEL HAVENS WINES
---------------------------------------
27000 RAMAL RD, SONOMA, CA 95476
Searching for: CARNEROS HILLS WINERY
---------------------------------------
Found result for 3384: CAR

22020 CARNEROS VINEYARD WAY STE 2, SONOMA, CA 95476
Searching for: MACLAREN WINE COMPANY
---------------------------------------
Found result for 3421: MACLAREN WINE COMPANY
---------------------------------------
64 BONNEAU RD, SONOMA, CA 95467
Searching for: ANABA WINES
---------------------------------------
Found result for 3422: ANABA WINES
---------------------------------------
21660 8TH ST E BLDG A, SONOMA, CA 95476
Searching for: FIGONE'S OF CALIFORNIA
---------------------------------------
Found result for 3423: FIGONE'S OF CALIFORNIA
---------------------------------------
21684 EIGHTH ST SUITE 470, SONOMA, CA 95476
Searching for: TALISMAN
---------------------------------------
Found result for 3424: TALISMAN
---------------------------------------
21684 EIGHTH ST STE 440 &, SONOMA, CA 95476
Searching for: TY CATON VINEYARDS
---------------------------------------
Found result for 3425: TY CATON VINEYARDS
---------------------------------------
21684 EIGHTH ST SUITE 460, S

Found result for 3461: MAGNOLIA BLOSSOM
---------------------------------------
No brewery found using addresss requirement.
Skipping....
1200 AMERICAN WAY, WINDSOR, CA 95492
Searching for: SMITH STORY WINE CELLARS
---------------------------------------
Found result for 3462: SMITH STORY WINE CELLARS
---------------------------------------
1200 AMERICAN WAY, WINDSOR, CA 95492
Searching for: SANGIACOMO FAMILY WINES
---------------------------------------
Found result for 3463: SANGIACOMO FAMILY WINES
---------------------------------------
4401 SLUSSER RD, WINDSOR, CA 95492
Searching for: SONOMA-CUTRER VINEYARDS
---------------------------------------
Found result for 3464: SONOMA-CUTRER VINEYARDS
---------------------------------------
7732 BELL RD, WINDSOR, CA 95492
Searching for: WEESE FAMILY WINES
---------------------------------------
Found result for 3465: WEESE FAMILY WINES
---------------------------------------
No brewery found using addresss requirement.
Skipping....


In [16]:
print(len(wine_name_yelp))
print(len(wine_biz_type))
print(len(wine_address))
print(len(wine_yelp_city))
print(len(wine_yelp_zip)) 
print(len(wine_lat)) 
print(len(wine_long)) 
print(len(wine_rating)) 
print(len(wine_rate_num))  


2690
2686
2686
2686
2686
2686
2686
2686
2686


In [23]:
#there is one more listing than it all the others? Probably due to error handling
#append general values, and check final csv
wine_biz_type.append("Wine of Something")
wine_address.append("general address")
wine_yelp_city.append("Somewhere in CA")
wine_yelp_zip.append(90000)
wine_lat.append(-0.23)
wine_long.append(0.23)
wine_rating.append(1.5)
wine_rate_num.append(1000)

In [24]:
print(len(wine_name_yelp))
print(len(wine_biz_type))
print(len(wine_address))
print(len(wine_yelp_city))
print(len(wine_yelp_zip)) 
print(len(wine_lat)) 
print(len(wine_long)) 
print(len(wine_rating)) 
print(len(wine_rate_num))  

2690
2690
2690
2690
2690
2690
2690
2690
2690


In [25]:
#make the dataframe for winery
yelp_winery_df = pd.DataFrame({
    "winery names":wine_name_yelp,
    "business type":wine_biz_type,
    "address": wine_address,
    "city": wine_yelp_city,
    "zip": wine_yelp_zip,
    "lat": wine_lat,
    "long": wine_long,
    "yelp rating": wine_rating,
    "number of yelp reviews": wine_rate_num  
})
yelp_winery_df.head()

,winery names,business type,address,city,zip,lat,long,yelp rating,number of yelp reviews
0,Dashe Cellars,Venues & Event Spaces,1951 Monarch St,Alameda,94501,37.780068,-122.310260,4.5,101
1,Ehrenberg Cellars,Wineries,5143 Tesla Rd,Livermore,94550,37.664772,-121.730827,4.0,35
2,Urban Legend Cellars,Wine Tasting Room,1951 Monarch St,Alameda,94501,37.780250,-122.310051,4.5,96
3,Building 43 Winery,Wineries,2440 Monarch St,Alameda,94501,37.785530,-122.308239,4.5,72
4,Freight & Salvage Coffeehouse,Music Venues,2020 Addison St,Berkeley,94704,37.870740,-122.269580,4.5,168


In [1]:
#record each iteration

yelp_winery_df.to_csv('winery_yelp_final2000.csv', sep='\t', encoding='utf-8')

NameError: name 'yelp_winery_df' is not defined

In [6]:
winery_csv = pd.read_csv('winery_yelp_final2000.csv', engine='python',sep='\t')
winery_csv.to_csv('winery_yelp.csv', encoding='utf-8')

winery_csv.head()

,Unnamed: 0,winery names,business type,address,city,zip,lat,long,yelp rating,number of yelp reviews
0,0,Dashe Cellars,Venues & Event Spaces,1951 Monarch St,Alameda,94501.0,37.780068,-122.310260,4.5,101
1,1,Ehrenberg Cellars,Wineries,5143 Tesla Rd,Livermore,94550.0,37.664772,-121.730827,4.0,35
2,2,Urban Legend Cellars,Wine Tasting Room,1951 Monarch St,Alameda,94501.0,37.780250,-122.310051,4.5,96
3,3,Building 43 Winery,Wineries,2440 Monarch St,Alameda,94501.0,37.785530,-122.308239,4.5,72
4,4,Freight & Salvage Coffeehouse,Music Venues,2020 Addison St,Berkeley,94704.0,37.870740,-122.269580,4.5,168
